In [1]:
%load_ext Cython

In [42]:
%%cython
from cython.parallel import prange
cdef int i
cdef int n = 1000000000
cdef long int sum = 0

for i in prange(n, nogil=True):
    sum += i

print(sum)

In [40]:
%%cython
import numpy as np
cimport numpy as np
import cython
from cython.parallel import prange
ctypedef np.double_t cDOUBLE
DOUBLE = np.float64

def mydot(np.ndarray[cDOUBLE, ndim=2] a, np.ndarray[cDOUBLE, ndim=2] b):
    cdef np.ndarray[cDOUBLE, ndim=2] c
    cdef int i, M, N, K

    c = np.zeros((a.shape[0], b.shape[1]), dtype=DOUBLE)
    M = a.shape[0]
    N = a.shape[1]
    K = b.shape[1]

    for i in prange(M, nogil=True, num_threads=10, schedule='guided'):
        multiply(&a[i,0], &b[0,0], &c[i,0], N, K)
    return c

@cython.wraparound(False)
@cython.boundscheck(False)
@cython.nonecheck(False)
cdef void multiply(double *a, double *b, double *c, int N, int K) nogil:
    cdef int j, k
    for j in range(N):
        for k in range(K):
            c[k] += a[j]*b[k+j*K]

In [27]:
import time
a = np.random.random((4000,500))
b = np.random.random((500,2000))

start=time.time()
#%timeit c = np.dot(a, b)
#print('finished dot: {} s'.format(time.clock()-t))
print('%.2f s'%(time.time()-start))
#t = time.clock()
start=time.time()
%timeit c2 = mydot(a, b)
print('%.2f s'%(time.time()-start))
#print('finished mydot: {} s'.format(time.clock()-t))

#print('Passed test:', np.allclose(c, c2))

0.00 s
3.35 s ± 53.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
26.86 s
